### Onboarding Waitlist 

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning

In [2]:
onboarding_data = pd.read_csv('onboarding_waitlist.csv', low_memory=False)
registration_data = pd.read_excel('network_register.xlsx', sheet_name='Sheet1')
network_waitlist_data = pd.read_excel('network_waitlist.xlsx', sheet_name='Sheet1')

print(len(onboarding_data))
print(len(registration_data))
print(len(network_waitlist_data))

257
503
275


### Prepare Data and Merge 

In [3]:
network_waitlist_data = network_waitlist_data[~network_waitlist_data["Waitlist No."].isnull()]
print(len(network_waitlist_data))

270


In [4]:
# Strip of leading and tailing white space
onboarding_data["safe_address"] = onboarding_data["safe_address"].str.strip()
registration_data["What is your HOPR safe address?"] = registration_data["What is your HOPR safe address?"].str.strip()
network_waitlist_data["Safe address"] = network_waitlist_data["Safe address"].str.strip()

# Make everything lower case letters
onboarding_data["safe_address"] = [x.lower() for x in onboarding_data["safe_address"]]
registration_data["What is your HOPR safe address?"] = [x.lower() for x in registration_data["What is your HOPR safe address?"]]
network_waitlist_data["Safe address"] = [x.lower() for x in network_waitlist_data["Safe address"]]

# Check for duplicates
dataDup_onboarding = onboarding_data.duplicated(subset=['safe_address'], keep='last')
dataDup_registration = registration_data.duplicated(subset=['What is your HOPR safe address?'], keep='last')

# dataDup.value_counts()
onboarding_data['Duplicate'] = dataDup_onboarding
registration_data['Duplicate'] = dataDup_registration

print(len(registration_data))

503


In [5]:
registration_data[registration_data["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,Time,Participant,What is your HOPR safe address?,What is your Node address,Do you already have the Network Registry NFT?,How would you like to be informed once you're able to join the network?,What is your Telegram handle?,What is your e-mail?,Duplicate
453,2023-10-02T19:23:47.388Z,landeros,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,Yes,I'll monitor the waitlist and my address manually,NaN,NaN,False


In [6]:
# Only keep unique values
onboarding_data_01 = onboarding_data.loc[onboarding_data['Duplicate'] == False]
registration_data_01 = registration_data.loc[registration_data['Duplicate'] == False]

print(onboarding_data_01['Duplicate'].value_counts())
print(registration_data_01['Duplicate'].value_counts())

Duplicate
False    257
Name: count, dtype: int64
Duplicate
False    476
Name: count, dtype: int64


In [12]:
# registration_data_01[registration_data_01["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]
onboarding_data_01[onboarding_data_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,deployment_date,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,nft_id,Duplicate
139,2023-10-02 18:49:25.000 UTC,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,18592.0,False


In [7]:
## Delete people who are already in the waitlist from the registration data

x = list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])
print(len(x))

registration_data_01['waitlist_ind'] = np.where(registration_data_01["What is your HOPR safe address?"].isin(x), "YES", "No")
registration_data_02 = registration_data_01[registration_data_01['waitlist_ind'] == "No"]
print(len(registration_data_02))

180
296


In [14]:
registration_data_02[registration_data_02["What is your HOPR safe address?"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,Time,Participant,What is your HOPR safe address?,What is your Node address,Do you already have the Network Registry NFT?,How would you like to be informed once you're able to join the network?,What is your Telegram handle?,What is your e-mail?,Duplicate,waitlist_ind
459,2023-10-02T19:23:47.388Z,landeros,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,Yes,I'll monitor the waitlist and my address manually,NaN,NaN,False,No


In [8]:
waitlist = registration_data_02.merge(onboarding_data_01, how='left', left_on='What is your HOPR safe address?'
                                                    , right_on='safe_address')

waitlist = waitlist[['Time', 'deployment_date', 'What is your Node address', 'safe_address', 'deployment_tx_hash', 'wxHOPR_balance', 'nr_nft']]

# rename comumns
waitlist = waitlist.rename(columns={"Time": "registration_time", "What is your Node address": "node_address"})
print(len(waitlist))

# exlude non elidgible nodes
waitlist_01 = waitlist[~waitlist['safe_address'].isnull()]
print(len(waitlist_01))
display(waitlist_01.head())

296
105


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
38,2023-09-09T14:31:55.044Z,2023-09-09 14:22:30.000 UTC,0x775175769897c0a46781b423c599f1b6a2b4cde8,0x244454ab29bafb5bbb95f8f6de28fcc45e81b620,0x506226a43ddfffc0bfb48124e7b782232624430010b1...,52186.000000,False
62,2023-09-09T15:50:12.825Z,2023-09-09 15:36:55.000 UTC,0x27d559ef0a9ab0d00dedbe5e0d5c215ec475ee24,0x4e4c24299bece3aff08b013d10023eb011394199,0xab810a4bac3fda9f2f5f6fa51818cdf0a7f9ad264216...,0.000037,False
64,2023-09-09T16:00:12.038Z,2023-09-09 13:32:15.000 UTC,0x9680ca3188117a40d6ce7c940020c7ac3afd58a4,0x865a0bef05fa813aae59ef12d88ace8eebef49d0,0x68a9e3e3c36c1b811beda8976d72e711d3fd156b9533...,0.000000,False
127,2023-09-10T07:07:53.959Z,2023-09-10 06:28:05.000 UTC,0xbd6e77e2b40e1613047479b0c7c720041d4e3f8f,0x138969cbb6b1346151986493bf82084eb98a5c2a,0x24203d9be19d0b0c109721eeb313afd9817cc16a2c45...,64257.000000,False
133,2023-09-10T09:21:04.773Z,2023-09-10 01:15:55.000 UTC,0x8930d5acfbcf11a317a25cb5df514fd64066caeb,0x76fae50243222dd030de8169d9f341448a363d13,0xe56122ca0eb702f8496c2ea7aad62e3b6c7b293e7e76...,50361.102173,False


In [16]:
waitlist_01[waitlist_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True


### Delete non-elidgible nodes 

In [9]:
# Not enough stake
waitlist_02 = waitlist_01[waitlist_01['wxHOPR_balance'] >= 10000]
print(len(waitlist_02))
print(waitlist_02.dtypes)

94
registration_time      object
deployment_date        object
node_address           object
safe_address           object
deployment_tx_hash     object
wxHOPR_balance        float64
nr_nft                 object
dtype: object


In [18]:
waitlist_02[waitlist_02["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True


In [10]:
waitlist_02.loc[:, 'nr_nft'] = waitlist_02['nr_nft'].astype(bool)
print(waitlist_02['nr_nft'].value_counts())

nr_nft
True     83
False    11
Name: count, dtype: int64


In [11]:
# Users with a stake >= 30000 are eligible regardless of NR NFT
waitlist_03 = waitlist_02.copy()
waitlist_03.loc[:, 'eligible'] = np.where(waitlist_02.loc[:,'wxHOPR_balance'] >= 30000, True, False)

# Users with a stake < 30000 are eligible if they have a network registry NFT
waitlist_04 = waitlist_03.copy()
waitlist_04.loc[:, 'eligible'] = np.where((waitlist_03.loc[:,'wxHOPR_balance'] < 30000) & (waitlist_03.loc[:,'nr_nft'] == True), True, waitlist_03.loc[:, 'eligible'])

print(waitlist_04.loc[:, 'eligible'].value_counts())


eligible
True    94
Name: count, dtype: int64


In [12]:
waitlist_04[waitlist_04["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
259,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [13]:
waitlist_05 = waitlist_04[waitlist_04.loc[:,'eligible'] == True]
print(len(waitlist_05))

94


In [23]:
waitlist_05[waitlist_05["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [14]:
waitlist_05["node_address"] = waitlist_05["node_address"].str.strip()

In [15]:
# delete users that registered with their peer_id
valid_address = waitlist_05['node_address'].str.startswith('0x')
waitlist_06 = waitlist_05[valid_address]
print(len(waitlist_06))

92


In [15]:
waitlist_06[waitlist_06["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
283,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [33]:
waitlist_06[waitlist_06["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
345,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


### Sort Users 

In [16]:
nr_waitlist = waitlist_06[waitlist_06['nr_nft'] == True]
stake_waitlist =waitlist_06[waitlist_06['nr_nft'] == False]

print(len(nr_waitlist))
print(len(stake_waitlist))

81
11


In [17]:
nr_waitlist_01 = nr_waitlist.sort_values(by="deployment_date", ascending=True).reset_index(drop=True)
print(len(nr_waitlist_01))
display(nr_waitlist_01.head())

81


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-18T01:46:36.412Z,2023-09-18 01:18:05.000 UTC,0x31faaa8ce2fe3a22eab7c05167312f853adc0042,0xcd1a97453e3525019cdfa66fb669576fafb2c527,0xd12a7eeb3e0e65b65e3c6940d85575d9b01c4e0a8af3...,158650.000,True,True
1,2023-09-18T07:12:37.024Z,2023-09-18 07:06:00.000 UTC,0x7cafae40005df7c759de08e88599852be15ce44c,0x726f8878703d6d2c25acfb2bfaf8585cdf695aef,0x69871c89338c401d48eb725de9e2ae8607b914bf7bac...,10005.000,True,True
2,2023-09-18T07:22:40.869Z,2023-09-18 07:15:45.000 UTC,0x6c7f48dd8d8b06ca6db651e3bfd841414bc8c0d0,0x1e2882047445776910af00ac90e5a968d3e4ed38,0x7849c3a255e429e24239b4047b54332bd6433846ab98...,10002.000,True,True
3,2023-09-25T08:56:29.295Z,2023-09-18 12:38:50.000 UTC,0x300280bbf63ecd05dc9b70dab395ab29a61b7433,0xedc612498b3b6718e73897207ecf050d83c3c09d,0x81ded857167d5cf49f3ce0ae27256546a4e6bab74915...,10004.744,True,True
4,2023-09-18T14:39:08.343Z,2023-09-18 14:34:35.000 UTC,0xdcf5cb3bd285dea6827bf84709698fa55a8a209d,0x3152e7539c677dc49a7530353ec5cbab02d3bd57,0xb1422f09adb7e21d241ab1239abb71f9e4f9e6af3807...,23942.000,True,True


In [18]:
nr_waitlist_01[nr_waitlist_01["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
78,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [19]:
print(len(nr_waitlist_01))

99


In [18]:
stake_waitlist_01 = stake_waitlist.sort_values(by="wxHOPR_balance", ascending=False).reset_index(drop=True)
print(len(stake_waitlist))
display(stake_waitlist_01.head())

11


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-10-03T11:23:06.563Z,2023-10-03 09:46:15.000 UTC,0x5c4d9d15aa67a7a06e595d006e4667bd140acb66,0x6504b06bf0aff6d85cd1eae6f54153a7bb72306f,0x8edc317a8d6d238bca5543ffc6bff454f1fd246c6a3e...,67951.000000,False,True
1,2023-09-10T07:07:53.959Z,2023-09-10 06:28:05.000 UTC,0xbd6e77e2b40e1613047479b0c7c720041d4e3f8f,0x138969cbb6b1346151986493bf82084eb98a5c2a,0x24203d9be19d0b0c109721eeb313afd9817cc16a2c45...,64257.000000,False,True
2,2023-09-09T14:31:55.044Z,2023-09-09 14:22:30.000 UTC,0x775175769897c0a46781b423c599f1b6a2b4cde8,0x244454ab29bafb5bbb95f8f6de28fcc45e81b620,0x506226a43ddfffc0bfb48124e7b782232624430010b1...,52186.000000,False,True
3,2023-09-10T09:21:04.773Z,2023-09-10 01:15:55.000 UTC,0x8930d5acfbcf11a317a25cb5df514fd64066caeb,0x76fae50243222dd030de8169d9f341448a363d13,0xe56122ca0eb702f8496c2ea7aad62e3b6c7b293e7e76...,50361.102173,False,True
4,2023-09-15T05:01:48.095Z,2023-09-14 08:19:40.000 UTC,0x4529c70c711b014f9e6f537812d4c30dc12fc2d3,0x5fb3844f2665db89105020955dbc23d2b5f389fb,0x5956c502d191960caf704f29a354a05f50b954b8a991...,45447.710557,False,True


In [19]:
print(len(stake_waitlist_01))

11


In [20]:
# Assuming you have two dataframes, nr_waitlist_01 and stake_waitlist_01
nr_chunk_size = 20
stake_chunk_size = 10

nr_index = 0  # Starting index for nr_waitlist_01
stake_index = 0  # Starting index for stake_waitlist_01

final_chunks = []  # List to store concatenated chunks

while nr_index < len(nr_waitlist_01) or stake_index < len(stake_waitlist_01):
    # Add a chunk of nr if there are more rows available
    if nr_index < len(nr_waitlist_01):
        nr_chunk = nr_waitlist_01.iloc[nr_index:nr_index + nr_chunk_size]
        final_chunks.append(nr_chunk)
        nr_index += nr_chunk_size

    # Add a chunk of stake if there are more rows available
    if stake_index < len(stake_waitlist_01):
        stake_chunk = stake_waitlist_01.iloc[stake_index:stake_index + stake_chunk_size]
        final_chunks.append(stake_chunk)
        stake_index += stake_chunk_size

# Concatenate the final chunks row-wise
final_waitlist = pd.concat(final_chunks, ignore_index=True)

display(final_waitlist.head())
print(len(final_waitlist))

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-18T01:46:36.412Z,2023-09-18 01:18:05.000 UTC,0x31faaa8ce2fe3a22eab7c05167312f853adc0042,0xcd1a97453e3525019cdfa66fb669576fafb2c527,0xd12a7eeb3e0e65b65e3c6940d85575d9b01c4e0a8af3...,158650.000,True,True
1,2023-09-18T07:12:37.024Z,2023-09-18 07:06:00.000 UTC,0x7cafae40005df7c759de08e88599852be15ce44c,0x726f8878703d6d2c25acfb2bfaf8585cdf695aef,0x69871c89338c401d48eb725de9e2ae8607b914bf7bac...,10005.000,True,True
2,2023-09-18T07:22:40.869Z,2023-09-18 07:15:45.000 UTC,0x6c7f48dd8d8b06ca6db651e3bfd841414bc8c0d0,0x1e2882047445776910af00ac90e5a968d3e4ed38,0x7849c3a255e429e24239b4047b54332bd6433846ab98...,10002.000,True,True
3,2023-09-25T08:56:29.295Z,2023-09-18 12:38:50.000 UTC,0x300280bbf63ecd05dc9b70dab395ab29a61b7433,0xedc612498b3b6718e73897207ecf050d83c3c09d,0x81ded857167d5cf49f3ce0ae27256546a4e6bab74915...,10004.744,True,True
4,2023-09-18T14:39:08.343Z,2023-09-18 14:34:35.000 UTC,0xdcf5cb3bd285dea6827bf84709698fa55a8a209d,0x3152e7539c677dc49a7530353ec5cbab02d3bd57,0xb1422f09adb7e21d241ab1239abb71f9e4f9e6af3807...,23942.000,True,True


92


In [21]:
print(len(final_waitlist))

92


In [44]:
final_waitlist[final_waitlist["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
125,2023-10-02T19:23:47.388Z,2023-10-02 18:49:25.000 UTC,0x5849b10e7889afc8f742b6efc94f65a5a1dddaa2,0xa1344345688ad2ab03c0423b0e1734b39b6d18d7,0xc079b6dac7167a416f9c7186a23071946d0d4cb9ae04...,10088.470606,True,True


In [22]:
### sanity checks

# check users who already got access are not in the waitlist
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])).value_counts())

# check that all users who were already in the waitlist are indeed in the waitlist
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'].isnull()]["Safe address"])).value_counts())

# check that no duplicates are in the waitlist
print((final_waitlist.duplicated(subset=['safe_address'], keep='last')).value_counts())

safe_address
False    92
Name: count, dtype: int64
safe_address
True     90
False     2
Name: count, dtype: int64
False    92
Name: count, dtype: int64


In [32]:
final_waitlist[final_waitlist["safe_address"] == "0xa1344345688ad2ab03c0423b0e1734b39b6d18d7"]

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible


### Save final waitlist 

In [23]:
final_waitlist.to_excel('final_waitlist.xlsx', index=False)